In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
class modeling:
  !pip install catboost
  # train과 test로 나누기
  def train_test_split(data = None):
    train = data[data['date'] <= 201712]
    test = data[data['date'] > 201712]

    X_train, y_train = train.drop('target', axis = 1), train['target']
    X_test, y_test = test.drop('target', axis = 1), test['target']

    return X_train, X_test, y_train, y_test

  # linear regression 돌리기
  def lr():
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    
    return model

  # xgboost 돌리기
  def xgb():
    from xgboost import XGBRegressor
    model = XGBRegressor(random_state = 42,tree_method = 'gpu_hist')

    return model

  # lightgbm
  def lgb():
    from lightgbm import LGBMRegressor
    model = LGBMRegressor(random_state = 42)

    return model

  # catboost
  def cat():
    from catboost import CatBoostRegressor
    model = CatBoostRegressor(random_state = 42,task_type = 'GPU')

    return model

  # ridge
  def ridge():
    from sklearn.linear_model import Ridge
    model = Ridge(random_state = 42)
    
    return model
  
  # lasso
  def lasso():
    from sklearn.linear_model import Lasso
    model = Lasso()

    return model

  # adaboost
  def ada():
    from sklearn.ensemble import AdaBoostRegressor
    model = AdaBoostRegressor(random_state = 42)

    return model

  # randomforest
  def rfg():
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor(random_state = 42)

    return model

  def rmse(y_test, y_pred):
    from sklearn.metrics import mean_squared_error
    RMSE = mean_squared_error(y_test, y_pred)**0.5

    return RMSE

# 모델 전부 한번씩 돌려보기
class class_all_model:
  # 각 모델을 돌리기 위해서 dict로 쌓아준다.
  def models():
    models = {'lr':modeling.lr(), 'lgb':modeling.lgb(), 'xgb':modeling.xgb(), 'lasso':modeling.lasso(),'ridge':modeling.ridge(),'cat':modeling.cat(),'ada':modeling.ada(), 'rfg':modeling.rfg()}
    return models
  
  # 모델을 하나씩 돌려서 keys = 모델명, value = 모델의 rmse를 쌓아준다.
  def run(data = None):
    models = class_all_model.models()
    final_models = {}
    model_score = {}
    model_binning = {}
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    # dict로 쌓아놓은 데이터를 전부 가져온다.
    for i, j in models.items():
      model = j
      model.fit(X_train, y_train)
      y_pred = model.predict(X_test)

      final_models[i] = modeling.rmse(y_test, y_pred)
      model_score[i] = model.score(X_test, y_test)
      model_binning[i] = model

    return final_models, model_score, model_binning

# stacking ensemble
class class_stack:
  def model_stack():
      models = [modeling.lgb(), modeling.cat(), modeling.lr(), modeling.xgb(), modeling.lasso(), modeling.ada(), modeling.rfg(),modeling.ridge()]
      return models
  def stack(data = None):
    !pip install vecstack
    from sklearn.metrics import mean_squared_error
    from vecstack import stacking
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)

    S_train, S_test = stacking(class_stack.model_stack(), X_train, y_train, X_test, regression = True, metric = ['acc'], n_folds = 3)
    model = modeling.lgb()
    model.fit(S_train, y_train)
    y_pred = model.predict(S_test)
    model_score = model.score(S_test, y_test)

    return modeling.rmse(y_test, y_pred), model_score, model
# voting ensemble
class class_voting:
  # 모델들을 voting에 맞게 모아준다.
  def collect_model():
    models= [('xgb',modeling.xgb()),('lgb',modeling.lgb()),('rfg',modeling.rfg()),('ada',modeling.ada()),('lr',modeling.lr()),('ridge',modeling.ridge()),('lasso',modeling.lasso()),('cat',modeling.cat())]
    
    return models

  # VotingRegressor를 돌려준다.
  def voting(data = None):
    from sklearn.ensemble import VotingRegressor
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    model = VotingRegressor(class_voting.collect_model(), )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_score = model.score(X_test, y_test)
    return modeling.rmse(y_test, y_pred), model_score, model

# class로 짠 모델들을 전부 돌려준다.
class final_model:
  def run(data = None):
    final_rmse , model_score, model = class_all_model.run(data = data)
    final_rmse['stack'], model_score['stack'], model['stack'] = class_stack.stack(data = data)
    final_rmse['voting'], model_score['voting'], model['voting'] = class_voting.voting(data = data)

    for i, j in final_rmse.items():
      if j == min(final_rmse.values()):
        print(j)
        model = model[i]

    return final_rmse, model_score, model

  def 
    print('------ 끝 --------------------------------------------------')

In [53]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Proj_WT/DataSets/박정열/덕수궁_0 (1).csv')
data.head()

,date,month,최고 기온(°C),최소 상대습도(%),평균 기온(°C),평균 상대습도(%),일강수량(mm),평균 풍속(m/s),합계 일조 시간(hr),0.5m 지중온도(°C),최대 풍속(m/s),합계 일사량(MJ/m2),target,여름철 체감온도,겨울철 체감온도,불쾌지수
0,201101,1,-3.406452,35.129032,-7.183871,53.829032,1.112500,2.796774,7.048387,1.541935,5.593548,9.951290,92135,219.748397,24.903816,28.955175
1,201102,2,5.864286,32.000000,1.221429,55.228571,9.700000,2.553571,5.950000,1.025000,5.239286,10.827857,132920,149.419352,34.268870,40.059503
2,201112,12,2.851613,31.677419,-0.870968,50.438710,0.591667,2.616129,6.309677,5.529032,5.141935,6.962258,52140,159.546683,31.939538,37.946869
3,201201,1,1.338710,30.548387,-2.787097,48.858065,0.558333,2.532258,6.145161,1.680645,5.341935,7.564516,40041,171.537869,29.780329,35.707644
4,201202,2,2.968966,23.620690,-2.044828,43.179310,0.133333,2.865517,7.755172,0.968966,5.641379,11.215517,36900,155.712604,30.660864,37.594935


In [54]:
data.describe()

,date,month,최고 기온(°C),최소 상대습도(%),평균 기온(°C),평균 상대습도(%),일강수량(mm),평균 풍속(m/s),합계 일조 시간(hr),0.5m 지중온도(°C),최대 풍속(m/s),합계 일사량(MJ/m2),target,여름철 체감온도,겨울철 체감온도,불쾌지수
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,201505.000000,5.000000,3.248742,32.645900,-1.151809,52.986949,2.327909,2.425819,6.350756,2.989766,4.973927,8.428650,74336.037037,164.775237,31.552288,37.202557
std,263.166078,5.061164,2.389827,4.869336,2.140673,4.909395,2.079229,0.361226,0.622069,1.782134,0.570589,1.908494,29537.493428,17.927582,2.385191,2.856361
min,201101.000000,1.000000,-3.406452,23.620690,-7.183871,43.179310,0.000000,1.782759,5.296774,0.560714,3.900000,4.893226,36900.000000,141.766935,24.903816,28.955175
25%,201301.500000,1.000000,1.827419,29.884793,-2.415962,49.467742,1.115625,2.202189,5.948387,1.762903,4.522753,6.992903,52468.500000,152.217042,30.098456,35.695601
50%,201502.000000,2.000000,3.525806,32.750000,-0.896774,53.664286,1.900000,2.477419,6.309677,2.578571,5.141935,7.952258,67840.000000,158.786461,31.872766,37.811563
75%,201707.000000,12.000000,5.175028,36.919355,0.589409,56.696774,2.785417,2.711118,6.736485,4.695161,5.450000,9.948576,96274.000000,173.470794,33.436796,39.218477
max,201912.000000,12.000000,6.603571,40.096774,1.928571,60.151613,9.700000,2.882143,7.755172,6.283871,5.741935,11.454815,132920.000000,219.748397,35.084261,41.421184


In [ ]:
/content/drive/MyDrive/Proj_WT/DataSets/박정열/덕수궁_가을 (1).csv

In [55]:
# class를 이용해서 모든 모델을 돌려준다.
final_rmse, model_score, model = final_model.run(data = data)

[10:19:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Learning rate set to 0.055526


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1412369988.5556505, tolerance: 1312106.386695238
  positive)


0:	learn: 24454.5750321	total: 7.75ms	remaining: 7.75s
1:	learn: 23980.0745223	total: 15.6ms	remaining: 7.8s
2:	learn: 23471.4814108	total: 24.4ms	remaining: 8.11s
3:	learn: 22945.2464296	total: 41.8ms	remaining: 10.4s
4:	learn: 22434.9246107	total: 46.8ms	remaining: 9.31s
5:	learn: 21887.1478531	total: 52.4ms	remaining: 8.68s
6:	learn: 21499.7263327	total: 58.6ms	remaining: 8.31s
7:	learn: 21050.6775731	total: 64.8ms	remaining: 8.03s
8:	learn: 20675.6806395	total: 71.3ms	remaining: 7.86s
9:	learn: 20134.3177306	total: 77.6ms	remaining: 7.68s
10:	learn: 19724.5550617	total: 83.9ms	remaining: 7.54s
11:	learn: 19377.0295822	total: 94.9ms	remaining: 7.81s
12:	learn: 18880.9065158	total: 101ms	remaining: 7.68s
13:	learn: 18547.3822463	total: 107ms	remaining: 7.57s
14:	learn: 18215.5010074	total: 114ms	remaining: 7.46s
15:	learn: 17857.8254742	total: 125ms	remaining: 7.67s
16:	learn: 17428.8418652	total: 131ms	remaining: 7.57s
17:	learn: 17064.1524339	total: 144ms	remaining: 7.83s
18:	learn

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Learning rate set to 0.054478
0:	learn: 19169.6703303	total: 7.13ms	remaining: 7.13s
1:	learn: 18737.8415589	total: 12.8ms	remaining: 6.38s
2:	learn: 18341.6495909	total: 18.4ms	remaining: 6.12s
3:	learn: 17953.8068864	total: 24ms	remaining: 5.98s
4:	learn: 17565.1665683	total: 29.4ms	remaining: 5.86s
5:	learn: 17264.5655938	total: 35ms	remaining: 5.79s
6:	learn: 16978.7066645	total: 42.2ms	remaining: 5.99s
7:	learn: 16569.4956902	total: 50.6ms	remaining: 6.27s
8:	learn: 16213.7499320	total: 56.1ms	remaining: 6.18s
9:	learn: 15946.4985498	total: 61.7ms	remaining: 6.1s
10:	learn: 15582.7890407	total: 67.2ms	remaining: 6.04s
11:	learn: 15223.4976270	total: 72.7ms	remaining: 5.98s
12:	learn: 14878.6543170	total: 78.3ms	remaining: 5.94s
13:	learn: 14660.5133412	total: 83.6ms	remaining: 5.89s
14:	learn: 14362.0681360	total: 89.1ms	remaining: 5.85s
15:	learn: 14131.4087459	total: 94.5ms	remaining: 5.81s
16:	learn: 13870.9810138	total: 106ms	remaining: 6.11s
17:	learn: 13671.5094788	total: 11

...

0:	learn: 29185.2230320	total: 5.5ms	remaining: 5.49s
1:	learn: 28507.7425072	total: 10ms	remaining: 5s
2:	learn: 27656.0755402	total: 14ms	remaining: 4.67s
3:	learn: 27005.0858279	total: 19.4ms	remaining: 4.83s
4:	learn: 26306.8361349	total: 25ms	remaining: 4.98s
5:	learn: 25637.8006634	total: 30.6ms	remaining: 5.07s
6:	learn: 25020.9994547	total: 41.6ms	remaining: 5.9s
7:	learn: 24386.7422530	total: 50.2ms	remaining: 6.22s
8:	learn: 23755.8614962	total: 55.7ms	remaining: 6.14s
9:	learn: 23200.2348757	total: 60.8ms	remaining: 6.02s
10:	learn: 22695.5264566	total: 66.4ms	remaining: 5.97s
11:	learn: 22168.8827994	total: 72ms	remaining: 5.93s
12:	learn: 21628.4367310	total: 77.5ms	remaining: 5.89s
13:	learn: 21209.4279845	total: 83.1ms	remaining: 5.86s
14:	learn: 20735.2962844	total: 88.2ms	remaining: 5.79s
15:	learn: 20222.4990401	total: 93.2ms	remaining: 5.73s
16:	learn: 19797.6765159	total: 98.8ms	remaining: 5.71s
17:	learn: 19393.1843108	total: 104ms	remaining: 5.7s
18:	learn: 19032.

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5070294.45975381, tolerance: 539336.3433214284
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10020691.937113943, tolerance: 810630.1235214287
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11631078.853671055, tolerance: 1251055.5243714286
  positive)


[10:19:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1412369988.5556505, tolerance: 1312106.386695238
  positive)


Learning rate set to 0.055526
0:	learn: 24454.5750321	total: 6.22ms	remaining: 6.22s
1:	learn: 23980.0745223	total: 12.6ms	remaining: 6.28s
2:	learn: 23471.4814108	total: 18.8ms	remaining: 6.24s
3:	learn: 22945.2464296	total: 27.5ms	remaining: 6.86s
4:	learn: 22434.9246107	total: 34.4ms	remaining: 6.84s
5:	learn: 21887.1478531	total: 40.1ms	remaining: 6.64s
6:	learn: 21499.7263327	total: 46.2ms	remaining: 6.56s
7:	learn: 21050.6775731	total: 52.4ms	remaining: 6.5s
8:	learn: 20675.6806395	total: 58.6ms	remaining: 6.45s
9:	learn: 20134.3177306	total: 64.8ms	remaining: 6.41s
10:	learn: 19724.5550617	total: 70.9ms	remaining: 6.38s
11:	learn: 19377.0295822	total: 77.1ms	remaining: 6.35s
12:	learn: 18880.9065158	total: 83.6ms	remaining: 6.35s
13:	learn: 18547.3822463	total: 89.8ms	remaining: 6.32s
14:	learn: 18215.5010074	total: 96.1ms	remaining: 6.31s
15:	learn: 17857.8254742	total: 102ms	remaining: 6.3s
16:	learn: 17428.8418652	total: 109ms	remaining: 6.28s
17:	learn: 17064.1524339	total: 

In [57]:
print(final_rmse, model_score, model)

{'lr': 67936.9404308032, 'lgb': 43249.31219833734, 'xgb': 46645.98934466342, 'lasso': 64917.50942882645, 'ridge': 66609.59382540888, 'cat': 42835.90585355409, 'ada': 41370.68654416333, 'rfg': 39415.24570836898, 'stack': 43249.31219833734, 'voting': 49397.928250825935} {'lr': -6.379015369879954, 'lgb': -1.990507229549183, 'xgb': -2.47868467336806, 'lasso': -5.7376763533522706, 'ridge': -6.0934909188941155, 'cat': -1.93360986274218, 'ada': -1.736351614202678, 'rfg': -1.4837902773617513, 'stack': -1.990507229549183, 'voting': -2.901251208998854} RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      ra

In [ ]:
# 더미화, 

In [6]:
class visualizer_class:

NameError: ignored

In [ ]:
import statsmodels.api as sm

train = data[data['date'] <= 201712]
test = data[data['date'] > 201712]




X_train, y_train = train.drop('target', axis = 1), train['target']
X_test, y_test = test.drop('target', axis = 1), test['target']


model = sm.OLS(y_train, X_train)
result = model.fit()
result.summary()

In [ ]:
y_test